<a href="https://colab.research.google.com/github/DianaOquendoEng/Proyecto-Kaggle-UDEA/blob/main/03-modelo_con_preprocesado_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



##MODELO SVM








In [24]:
#IMPORTAMOS LAS LIBRERIAS NECESARIAS
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report



In [25]:
#CARGAR LOS CSV PREPROCESADOS
train = pd.read_csv("train_preprocesado.csv")
test  = pd.read_csv("test_preprocesado.csv")

print(train.shape, test.shape)


(692500, 21) (296786, 1024)


In [27]:
# VALIDAR QUE EXISTA LA COLUMNA TARGET
print(train.columns)

if "target" not in train.columns:
    raise ValueError("ERROR: El CSV de train NO tiene la columna 'target'.")


Index(['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4', 'target'],
      dtype='object')


In [28]:
#REDUCIR LA MEMORIA DE LOS ARCHIVOS PREPROCESADOS
def reducir_memoria(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type == "int64":
            df[col] = df[col].astype("int32")

        elif col_type == "float64":
            df[col] = df[col].astype("float32")

        elif col_type == "object":
            # Convertir textos a category para ahorrar memoria
            df[col] = df[col].astype("category")

    return df

train = reducir_memoria(train)
test  = reducir_memoria(test)


In [29]:
# SEPARAR X_TRAIN y Y_TRAIN
X_train = train.drop(columns=["target"])
y_train = train["target"]

X_test = test.copy()

print("Columnas de X_train:", X_train.shape)


Columnas de X_train: (692500, 20)


In [30]:
# CONSERVAR SOLO COLUMNAS NUMERIAS
X_train = X_train.select_dtypes(include=["int32","float32"])
X_test  = X_test.select_dtypes(include=["int32","float32"])

print("Columnas numéricas:", X_train.shape[1])


Columnas numéricas: 5


In [31]:
# EVALUAR EL MODELO CON EL ACCURACY
from sklearn.model_selection import cross_val_score

# Usamos el mismo modelo que entrenas después
modelo_eval = LinearSVC(
    C=1.0,
    max_iter=5000,
    dual=False
)

scores = cross_val_score(modelo_eval, X_train, y_train, cv=5, scoring="accuracy")

print("Resultados cross-validation:", scores)
print("Accuracy promedio:", scores.mean())

Resultados cross-validation: [0.33051986 0.32958845 0.33197834 0.32847653 0.33024549]
Accuracy promedio: 0.3301617328519856


In [32]:
# ENTRENAR EL MODELO SVM LIVIANO
modelo = LinearSVC(
    C=1.0,
    max_iter=5000,
    dual=False
)

modelo.fit(X_train, y_train)
print("Modelo entrenado correctamente.")


Modelo entrenado correctamente.


In [33]:
# CREAR PREDICCIONES
predicciones = modelo.predict(X_test)
predicciones[:10]


array([3, 1, 3, 0, 0, 0, 3, 3, 0, 1], dtype=int32)

In [34]:
# GUARDAR PREDICCIONES
pd.DataFrame({"pred": predicciones}).to_csv("predicciones_svm.csv", index=False)
print("Archivo predicciones_svm.csv guardado.")

Archivo predicciones_svm.csv guardado.


In [35]:
predicciones = modelo.predict(X_test)


In [36]:

# CREAR ARCHIVO DE SUBMISSION PARA KAGGLE
import numpy as np
import pandas as pd

# Crear IDs consecutivos (Kaggle los necesita)
ids = np.arange(len(predicciones))

submission = pd.DataFrame({
    "id": ids,
    "target": predicciones
})

submission.to_csv("submission.csv", index=False)

print("Archivo 'submission.csv' creado correctamente.")
submission.head()


Archivo 'submission.csv' creado correctamente.


,id,target
0,0,3
1,1,1
2,2,3
3,3,0
4,4,0
